<a href="https://colab.research.google.com/github/esheshka/team_one_project/blob/Andrey/%D0%A7%D1%82%D0%BE_%D1%82%D0%BE_%D0%B7%D0%B0%D1%87%D0%B5%D0%BC_%D1%82%D0%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Imports**

In [839]:
from google.colab import drive; drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [840]:
!pip -q install --upgrade --force-reinstall --no-deps kaggle > log  # upgrade kaggle package (to avoid a warning)
!mkdir -p ~/.kaggle                                           # .kaggle folder must contain kaggle.json for kaggle executable to properly authenticate you to Kaggle.com
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json >log  # First, download kaggle.json from kaggle.com (in Account page) and place it in the root of mounted Google Drive
!cp kaggle.json ~/.kaggle/kaggle.json > log                   # Alternative location of kaggle.json (without a connection to Google Drive)
!chmod 600 ~/.kaggle/kaggle.json                              # give only the owner full read/write access to kaggle.json
!kaggle config set -n competition -v 26jun23                  # set the competition context for the next few kaggle API calls. !kaggle config view - shows current settings
!kaggle competitions download >> log                          # download competition dataset as a zip file
!unzip -o *.zip >> log                                        # Kaggle dataset is copied as a single file and needs to be unzipped.
!kaggle competitions leaderboard --show                       # print public leaderboard

cp: cannot stat 'kaggle.json': No such file or directory
- competition is now set to: 26jun23
Using competition: 26jun23
  teamId  teamName                   submissionDate       score    
--------  -------------------------  -------------------  -------  
10612701  Никита Угрюмов             2023-06-28 23:59:12  0.87976  
10612695  Ne1tr0N [Maxim Mashtaler]  2023-06-28 22:04:00  0.87931  
10612706  Michael Kondrashov         2023-06-28 14:52:04  0.87746  
10616508  Ulyana Klushova            2023-06-28 20:38:57  0.87485  
10612698  Alexandr Boldirev          2023-06-28 21:14:30  0.87344  
10614521  Konstantin Zeigman         2023-06-29 11:43:59  0.87169  
10612715  Stepan Vysotsky            2023-06-28 17:57:40  0.87076  
10614699  Dmitriy Geyzha             2023-06-28 12:50:43  0.86653  
10613713  Mikhail Vovk               2023-06-28 15:19:07  0.86403  
10617148  Vladimir Babushkin         2023-06-29 08:20:50  0.85571  
10612680  esheshka                   2023-06-29 09:39:16  0.854

In [841]:
%%time
%%capture
%reset -f
!pip install -U sentence-transformers tfds-nightly tensorflow tensorflow_addons plotly > log
from IPython.core.interactiveshell import InteractiveShell as IS; IS.ast_node_interactivity = "all"  # allows multiple prints from a cell
import numpy as np, pandas as pd, time, matplotlib.pyplot as plt, os, plotly.express as px, xgboost as xgb, lightgbm, re, tensorflow as tf, tensorflow.keras as keras
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GroupKFold
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPClassifier   # SKLearn's MLP is optimised for CPU (and doesn't use GPU)
from sentence_transformers import SentenceTransformer as SBERT
from keras.layers import Flatten, Dense

os.environ['TF_DETERMINISTIC_OPS'] = '1'; os.environ['TF_CUDNN_DETERMINISTIC'] = '1'   # helps to seed TF's RNG on GPUs
np.set_printoptions(linewidth=10000, precision=2, edgeitems=20, suppress=True)
pd.set_option('display.max_colwidth', 100, 'display.max_columns', 10, 'display.width', 1000, 'display.max_rows', 8)

class Timer():
  def __init__(self, lim:'RunTimeLimit'=60*2): self.t0, self.lim, _ = time.time(), lim, print(f'⏳ started. You have {lim} sec. Good luck!')
  def ShowTime(self):
    msg = f'Runtime is {time.time()-self.t0:.0f} sec'
    print(f'\033[91m\033[1m' + msg + f' > {self.lim} sec limit!!!\033[0m' if (time.time()-self.t0-1) > self.lim else msg)

CPU times: user 1.43 s, sys: 48.2 ms, total: 1.48 s
Wall time: 22.7 s


In [842]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier as SC
from sklearn.neighbors import KNeighborsClassifier as KNC

from sklearn.model_selection import GridSearchCV as GSCV
from sklearn.model_selection import RandomizedSearchCV as RSCV


In [843]:
#@title MCC
%time mcc = pd.read_csv('_mcc.csv', index_col='mcc')

CPU times: user 5.43 ms, sys: 1 ms, total: 6.43 ms
Wall time: 14.4 ms


In [844]:
#@title TTC
%time ttc = pd.read_csv('_ttc.csv', index_col='ttc');

CPU times: user 5.36 ms, sys: 0 ns, total: 5.36 ms
Wall time: 23 ms


In [845]:
#@title Gender
%time gender = pd.read_csv('gender.csv', index_col='cid'); gender.T
trainY, testY = gender.query('gender==gender'), gender.query('gender!=gender')

CPU times: user 8.23 ms, sys: 0 ns, total: 8.23 ms
Wall time: 25.1 ms


cid,0,1,2,3,4,...,8395,8396,8397,8398,8399
gender,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,0.0


In [846]:
#@title TRX
%time dfTrx = pd.read_csv('trx.csv').sort_values(['cid','dt','tm'])

CPU times: user 9.42 s, sys: 454 ms, total: 9.87 s
Wall time: 13.5 s


# **Тесты**

In [985]:
df_test = dfTrx.join(gender, on='cid', how='inner')
df_test = df_test[~df_test.gender.isna()]

In [1012]:
mcc_list = list(set(dfTrx.mcc))
df_mcc = pd.DataFrame({'mcc': mcc_list})

In [1034]:
zero = df_test[df_test.gender == 0][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
zero = df_mcc.join(pd.DataFrame({'zero': zero}), on='mcc', how='outer').fillna(0)
one = df_test[df_test.gender == 1][['cid', 'mcc']].drop_duplicates().groupby('mcc').agg('size').fillna(0)
one = df_mcc.join(pd.DataFrame({'one': one}), on='mcc', how='outer').fillna(0)

In [1038]:
df_mcc = df_mcc.join(pd.DataFrame({'count': (zero.zero + one.one)}), on='mcc', how='left').join(pd.DataFrame({'%': zero.zero / (zero.zero + one.one) * 100}), on='mcc', how='left')


# **My Code**

In [952]:
#@title Настройки

large_count = 2000
large_sum = -1000000000

k1 = 0.85
k2 = 0.85
# k3 = 1.25
k3 = 8

In [953]:
#@title Определение наиболее используемых mcc
%%time
mcc_by_cid = dfTrx[['cid', 'mcc']].drop_duplicates().astype({'mcc': str}).groupby('cid').agg({'mcc': ' '.join}).sort_values('cid')

large_mcc_count = dfTrx[['cid', 'mcc']].drop_duplicates().groupby('mcc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_mcc_sum = dfTrx[['amt', 'mcc']].groupby('mcc').agg('sum').sort_values(by = 'amt')

most_popular_mcc = set(large_mcc_count[large_mcc_count.cid > large_count * k1].index.to_list()) | set(large_mcc_sum[large_mcc_sum.amt < large_sum * k1].index.to_list()) | set([123, 125, 130])
len(most_popular_mcc)

CPU times: user 700 ms, sys: 1.97 ms, total: 702 ms
Wall time: 723 ms


35

In [954]:
#@title Определение наиболее используемых ttc
%%time
ttc_by_cid = dfTrx[['cid', 'ttc']].drop_duplicates().astype({'ttc': str}).groupby('cid').agg({'ttc': ' '.join}).sort_values('cid')

large_ttc_count = dfTrx[['cid', 'ttc']].drop_duplicates().groupby('ttc').agg({'cid': 'size'}).sort_values(by = 'cid')
large_ttc_sum = dfTrx[['amt', 'ttc']].groupby('ttc').agg('sum').sort_values(by = 'amt')

most_popular_ttc = set(large_ttc_count[large_ttc_count.cid > large_count * k2].index.to_list()) | set(large_ttc_sum[(large_ttc_sum.amt < large_sum * k2) | (large_ttc_sum.amt > 1000000000)].index.to_list())
len(most_popular_ttc)

CPU times: user 630 ms, sys: 3 ms, total: 633 ms
Wall time: 643 ms


25

In [955]:
#@title Определение наиболее используемых tid
%%time
tid_by_cid = dfTrx[['cid', 'tid']].drop_duplicates().astype({'tid': str}).groupby('cid').agg({'tid': ' '.join}).sort_values('cid')

large_tid_count = dfTrx[['cid', 'tid']].drop_duplicates().groupby('tid').agg({'cid': 'size'}).sort_values(by = 'cid')
large_tid_sum = dfTrx[['amt', 'tid']].groupby('tid').agg('sum').sort_values(by = 'amt')

most_popular_tid = set(large_tid_count[large_tid_count.cid > large_count * k3].index.to_list()) | set(large_tid_sum[(large_tid_sum.amt < large_sum * k3) | (large_tid_sum.amt > 1000000000)].index.to_list())
len(most_popular_tid)

CPU times: user 4.06 s, sys: 23.1 ms, total: 4.08 s
Wall time: 4.1 s


0

# **Начало создания df по cid**

In [956]:
#@title Сумма

def s(to_df):
    return to_df / to_df.sum(axis=0)
    # return to_df / np.sqrt((to_df**2).sum(axis=0))
    # return to_df / (np.sqrt(to_df**2)).sum(axis=0)

In [957]:
#@title Настройки

stat = ['count', 'sum', 'median']
is_PCA = 0

In [958]:
%%time
df = df_.copy(deep=True)

CPU times: user 239 µs, sys: 0 ns, total: 239 µs
Wall time: 247 µs


In [959]:
#@title MCC
%%time
to_df = df_.copy(deep=True)

for mcc_index in most_popular_mcc:
    stat_amt_by_cid_and_mcc = dfTrx[dfTrx.mcc == mcc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_mcc.columns += '_' + str(mcc_index) + '_mcc'
    to_df = to_df.join(stat_amt_by_cid_and_mcc, how='left')

stat_by_other_mcc = dfTrx[~dfTrx.mcc.isin(most_popular_mcc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_mcc.columns += '_mcc'
to_df = to_df.join(stat_by_other_mcc, how='left')

to_df = s(to_df) * 75
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.59 s, sys: 10 ms, total: 1.6 s
Wall time: 1.61 s


In [960]:
#@title TTC
to_df = df_.copy(deep=True)

for ttc_index in most_popular_ttc:
    stat_amt_by_cid_and_ttc = dfTrx[dfTrx.ttc == ttc_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_ttc.columns += '_' + str(ttc_index) + '_ttc'
    to_df = to_df.join(stat_amt_by_cid_and_ttc, how='left')

stat_by_other_ttc = dfTrx[~dfTrx.ttc.isin(most_popular_ttc)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_ttc.columns += '_ttc'
to_df = to_df.join(stat_by_other_ttc, how='left')

to_df = s(to_df) * 1/2
df = df.join(to_df, on = 'cid', how='left')

In [961]:
#@title AMT

to_df = df_.copy(deep=True)

to_df = dfTrx[['cid', 'amt']].groupby('cid').agg(stat).add_prefix('').droplevel(0, axis='columns')
to_df = df_.join(to_df, how='left').fillna(0)

to_df = s(to_df) * 2/3
df = df.join(to_df, on = 'cid', how='left')

In [962]:
#@title Time
%%time
to_df = df_.copy(deep=True)

for hour in range(0, 1):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df) * 2
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(1, 9):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for hour in range(9, 24):
    stat_amt_by_cid_and_hour = dfTrx[dfTrx.tm == hour][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_hour.columns += '_' + str(hour) + '_hour'
    to_df = to_df.join(stat_amt_by_cid_and_hour, how='left')

# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 13.8 s, sys: 82.7 ms, total: 13.9 s
Wall time: 14.1 s


In [963]:
#@title Date
%%time
to_df = df_.copy(deep=True)

# for weekday in range(7):
#     sum_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg({'amt': 'sum'}).sort_values(by = 'cid')
#     to_df = to_df.join(sum_amt_by_cid_and_weekday.amt, on = 'cid', how='left').rename(columns = {'amt': str(weekday) + '_wd'})

for weekday in [0, 3, 4, 5, 6]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')
# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 2), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
# to_df = pd.DataFrame(to_df.sum(axis=1))
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')
to_df = df_.copy(deep=True)

for weekday in [1, 2]:
    stat_amt_by_cid_and_weekday = dfTrx[dfTrx.dt % 7 == weekday][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_weekday.columns += '_' + str(weekday) + '_wd'
    to_df = to_df.join(stat_amt_by_cid_and_weekday, on = 'cid', how='left')


# to_df = pd.DataFrame(PCA(n_components=(to_df.shape[1] // 3), random_state=0).fit_transform(to_df.fillna(0).astype('int')))
# to_df.columns += df.shape[1]
to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

CPU times: user 1.25 s, sys: 1.02 ms, total: 1.25 s
Wall time: 1.24 s


In [964]:
#@title TID
to_df = df_.copy(deep=True)

for tid_index in most_popular_tid:
    stat_amt_by_cid_and_tid = dfTrx[dfTrx.tid == tid_index][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
    stat_amt_by_cid_and_tid.columns += '_' + str(tid_index) + '_tid'
    to_df = to_df.join(stat_amt_by_cid_and_tid, how='left')

stat_by_other_tid = dfTrx[~dfTrx.tid.isin(most_popular_tid)][['cid', 'amt']].groupby('cid').agg(stat).sort_values(by = 'cid').add_prefix('').droplevel(0, axis='columns')
stat_by_other_tid.columns += '_tid'
to_df = to_df.join(stat_by_other_tid, how='left')

to_df = s(to_df)
df = df.join(to_df, on = 'cid', how='left')

In [965]:
df = df.fillna(0).astype('float')
df.columns = [str(c) for c in df.columns]

# **Тест**

# **Обучение**

In [966]:
%%time
df.columns = [str(c) for c in df.columns]

trainX, testX = df.join(trainY.drop('gender', axis=1), how='inner'), df.join(testY.drop('gender', axis=1), how='inner') # split observations into train/test
trainX0, testX0, trainY0, testY0 = train_test_split(trainX, trainY, test_size=0.3, random_state=0) # split train observation into local train/test sets (to score performance)

# estimators = [
#     ('rfc', RFC(n_estimators=10, random_state=42)),
#     # ('knc', KNC()),
#     # ('svr', make_pipeline(StandardScaler(),
#     ('lr', LR(random_state=0, n_jobs=-1, max_iter=1000)),
#     # ('svc', SVC(probability=True)),
#     # ('gbc0', GBC()),
#     ('gbc1', GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2))
# ]
# model = SC(estimators=estimators, final_estimator=GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2))

# model = LR(random_state=0, n_jobs=-1, max_iter=1000)                             # always seed your experiments for reproducibility
# model = RFC()
# model = KNC()
# params = {
#     'n_neighbors': np.arange(1, 11),
#     'metric': ['manhattan', 'euclidean'],
#     'weights': ['uniform', 'distance']
# }
# model = DTC()
# model = SVC(probability=True)
# model = GBC()
model = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
# params = {
#     'max_depth': [None, 2, 3, 4, 5],
# }

# model = GSCV(model, params, return_train_score=True, cv=5, scoring='accuracy', n_jobs=-1)
pY = model.fit(trainX0, trainY0.gender).predict_proba(testX0)[:, 1]
# print(model.best_params_)

print('Local AUC ROC', roc_auc_score(testY0.gender, pY))       # fit the model and show test error
print(pY.sum() / pY.shape[0])

Local AUC ROC 0.8552364417989416
0.43340474555133474
CPU times: user 20 s, sys: 41.5 ms, total: 20 s
Wall time: 21 s


In [276]:
tmr.ShowTime()

Runtime is 1717 sec > 120 sec limit!!!


In [735]:
# model = LR(random_state=0, n_jobs=-1, max_iter=1000).fit(trainX, trainY.gender)  # now train again, on the full training set
# model = GBC(loss='log_loss', learning_rate=0.1, n_estimators=150, subsample=1.0, criterion='friedman_mse', max_depth=2)
model = SC(estimators=estimators)
model.fit(trainX, trainY.gender)

StackingClassifier(estimators=[('knc', KNeighborsClassifier()),
                               ('gbc1',
                                GradientBoostingClassifier(max_depth=2,
                                                           n_estimators=150))])

In [736]:
pY = pd.DataFrame(model.predict_proba(testX)[:,1], columns=['gender'])
pY.index.name = 'cid'

print(pY.sum() / pY.shape[0])

gender    0.443865
dtype: float64


In [737]:
pY.reset_index().to_csv('💳Baseline.csv', index=False, float_format='%10f')